In [1]:
import pandas as pd
import numpy as np
#import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')
from functions import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
# Import MultinomialNB
from sklearn.naive_bayes import MultinomialNB

### SIMPLE RECOMMENDATION ENGINE WITH COUNT VECTORIZER
### 2 NGRAMS FOR DESCRIPTION

In [6]:
train = pd.read_csv('../datasets/henry_lab_disk/properties_colombia_train.csv', sep = ',')
train.drop('id', axis=1, inplace=True)
train.rename(columns={'Unnamed: 0':'id'}, inplace=True)
train.set_index('id', inplace=True)

In [7]:
train['target'] = np.where(train['price'] > train['price'].mean(), 1, 0)
train = train[['title', 'description', 'target']]
train['target'].fillna(0, inplace=True)
train['description'].fillna(' ', inplace=True)
train['title'].fillna(' ', inplace=True)
train = trim_all_columns(train)
train['title'] = normalize_column(train, 'title')
train['description'] = normalize_column(train, 'description')
train['title'] = train['title'].str.lower().str.strip()
train['description'] = train['description'].str.lower().str.strip()
pattern = '|'.join(['\n','\r', '\t' ,'\xa0','\u200b',','])
train['title'] = clean_values(train['title'], pattern, value=' ')
train['description'] = clean_values(train['description'], pattern, value=' ')
pattern2 = '|'.join(['_', '[(|)]', '-',':',';'])
train['title'] = clean_values(train['title'], pattern2, regex = True, value=' ')
train['description'] = clean_values(train['description'], pattern2, regex = True, value=' ')
train['title'] = clean_values(train['title'], r"\<.*?\>", regex = True, value=' ')
train['description'] = clean_values(train['description'], pattern2, regex = True, value=' ')
train['title'] = clean_values(train['title'], r"\{.*?\}", regex = True, value=' ')
train['description'] = clean_values(train['description'], pattern2, regex = True, value=' ')
train['title'] = train['title'].str.replace(' +',' ', regex=True)
train['description'] = train['description'].str.replace(' +',' ', regex = True)
train['description'] = train['description'].str.replace('br / ','',regex = False)
train['description'] = train['description'].str.replace('/b','',regex = False)
train['description'] = train['description'].str.replace(' br ','',regex = False)
train['description'] = train['description'].str.replace(' b ','',regex = False)
train['description'] = train['description'].str.replace('&aacute ','a',regex = False)
train['description'] = train['description'].str.replace('&eacute ','e',regex = False)
train['description'] = train['description'].str.replace('&iacute ','i',regex = False)
train['description'] = train['description'].str.replace('&oacute ','o',regex = False)
train['description'] = train['description'].str.replace('&uacute ','u',regex = False)
train['description'] = train['description'].str.replace('&ntilde ','ñ',regex = False)
train['description'] = train['description'].str.replace('ref#\d+','',regex = True)
train['description'] = train['description'].str.replace('!!!','',regex = False)

In [72]:
# Create CountVectorizer object
vectorizer = CountVectorizer(strip_accents='ascii', stop_words=stopwords, lowercase=False, ngram_range=(1,2))

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(train['description'], 
train['target'], test_size=0.2,stratify=train['target'], random_state = 1234)

# Generate training Bow vectors
X_train_bow = vectorizer.fit_transform(X_train)
# Generate test BoW vectors
X_test_bow = vectorizer.transform(X_test)

# Create MultinomialNB object
clf = MultinomialNB()
# Train clf
clf.fit(X_train_bow, y_train)
# Compute accuracy on test set
accuracy = clf.score(X_test_bow, y_test)

print("The accuracy of the classifier on the test set is %.3f" % accuracy)

/Users/maico/Henry/DeepLearning/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:394: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['eramos', 'estabamos', 'estais', 'estan', 'estara', 'estaran', 'estaras', 'estare', 'estareis', 'estaria', 'estariais', 'estariamos', 'estarian', 'estarias', 'esteis', 'esten', 'estes', 'estuvieramos', 'estuviesemos', 'fueramos', 'fuesemos', 'habeis', 'habia', 'habiais', 'habiamos', 'habian', 'habias', 'habra', 'habran', 'habras', 'habre', 'habreis', 'habria', 'habriais', 'habriamos', 'habrian', 'habrias', 'hayais', 'hubieramos', 'hubiesemos', 'mas', 'mia', 'mias', 'mio', 'mios', 'seais', 'sera', 'seran', 'seras', 'sere', 'sereis', 'seria', 'seriais', 'seriamos', 'serian', 'serias', 'si', 'tambien', 'tendra', 'tendran', 'tendras', 'tendre', 'tendreis', 'tendria', 'tendriais', 'tendriamos', 'tendrian', 'tendrias', 'teneis', 'tengais', 'tenia', 'teniais', 'teniamos', 'teni

MultinomialNB()

The accuracy of the classifier on the test set is 0.900


In [53]:
test = pd.read_csv('../datasets/henry_lab_disk/properties_colombia_test.csv', sep = ',')
test.drop('id', axis=1, inplace=True)
test.rename(columns={'Unnamed: 0':'id'}, inplace=True)
test.set_index('id', inplace=True)
test = test[['title', 'description']]
test['description'].fillna(' ', inplace=True)
test['title'].fillna(' ', inplace=True)

In [80]:
# Predict the sentiment of a expensive sale
review1 = test['description'].sample().values
prediction = clf.predict(vectorizer.transform([str(review1)]))[0]
print("The sentiment predicted by the classifier is %i" % (prediction))
review1

The sentiment predicted by the classifier is 0


array(['OPORTUNIDAD| CASA LOTE EN VENTA Consta de. Sala comedor| cocina convencional| baño| patio. Opción de construir a gusto| terreno ideal  para uso comercial o bodega.Entorno Relevante. Centro comercial Centro Suba| Cai de Aures| D1| cerca a  vía principal| Parque Gloria Lara| Colegios cercanos.Barrios Colindantes. La Estanzuela| Aures| El Rosal| Lagos de Suba| San Jorge.'],
      dtype=object)

In [81]:
# Predict the sentiment of a expensive sale
review2 = test['description'].sample().values
prediction = clf.predict(vectorizer.transform([str(review2)]))[0]
print("The sentiment predicted by the classifier is %i" % (prediction))
review2

The sentiment predicted by the classifier is 1


array(['Hermosa Casa en Venta, diseñada y ubicada estratégicamente dentro de un lote de 1.303mt2, construcción de estilo rustico de un piso  mas guardilla que tiene salida a terraza superior, área construida de 150mt2. Tiene tres alcobas, la principal con baño, vestier y salida a una de las tres  terrazas que tiene esta acogedora casa. Cocina integral abierta, sala y comedor con con salida a otra terraza y zona de asados. Amplia e iluminada su diseño permite conexión constante con la naturaleza por sus amplios ventanales con vista a las montañas y a sus zonas verdes con arboles frutales, lote tiene pozo de agua, garaje hasta para 4 vehículos y deposito. El condominio tiene club house, cancha de tenis, microfútbol y cancha múltiple, piscina, jacuzzi, senderos ecológicos, 6 lagos y bosque nativo. Tan solo a 4 minutos de la 14 de alfaguara y 15min de Cali. !Ven y conoce tu próximo hogar!'],
      dtype=object)

In [86]:
test.shape[0]

65850

In [94]:
predictions = []
for i in range(test.shape[0]):
    prediction = clf.predict(vectorizer.transform([str(test.iloc[i,1])]))[0]
    predictions.append(prediction)

len(predictions)

65850

In [97]:
df = pd.DataFrame(predictions, columns=['target'])
df.to_csv('./data/predictions/predictions_nlp_recommendation.csv', index=False)

# NEURAL NETWORK WITH SPACY

NEW METHOD 3.4

In [42]:
import pandas as pd
import numpy as np
from spacy import displacy
from spacy.util import minibatch, compounding
from typing import Set, List, Tuple
from spacy.tokens import DocBin
#import spacy
# Para instalar modulos: https://spacy.io/usage/models
nlp = spacy.load('es_core_news_lg')
import matplotlib.pyplot as plt
%matplotlib inline

# Create a function to create a spacy dataset
def make_docs(data: List[Tuple[str, str]], target_file: str, cats: Set[str]):
    docs = DocBin()
    # Use nlp.pipe to efficiently process a large number of text inputs, 
    # the as_tuple arguments enables giving a list of tuples as input and 
    # reuse it in the loop, here for the labels
    for doc, label in nlp.pipe(data, as_tuples=True):
        # Encode the labels (assign 1 the subreddit)
        for cat in cats:
            doc.cats[cat] = 1 if cat == label else 0
        docs.add(doc)
    docs.to_disk(target_file)
    return docs

In [41]:
#cats = train.target.unique().tolist()
cats = ['1','0']

In [34]:
train.drop_duplicates(inplace=True)
train.drop_duplicates(subset='description', inplace=True)
train.shape

(109146, 4)

In [43]:
train_s = train.sample(frac = 0.35, random_state=42)

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_s["description"].values, train_s["target"].values, test_size=0.3)
make_docs(list(zip(X_train, y_train)), "train.spacy", cats=cats)
make_docs(list(zip(X_valid, y_valid)), "valid.spacy", cats=cats)

https://medium.com/@ycouble/training-and-integrating-a-custom-text-classifier-to-a-spacy-pipeline-b19e6a132487

In [69]:
from spacy.cli.train import train as spacy_train
spacy.prefer_gpu()
config_path = "../notebooks/configs/config.cfg"
output_model_path = "../notebooks/assets/model"
spacy_train(
    config_path,
    output_path=output_model_path,
    overrides={
        "paths.train": "train.spacy",
        "paths.dev": "valid.spacy",
    },
)

ℹ Saving to output directory: ../notebooks/assets/model
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.25       50.00    0.50
  0     200         50.00       50.00    0.50
  0     400         50.00       50.00    0.50
  0     600         50.00       50.00    0.50


KeyboardInterrupt: 

In [66]:
prueba1 = test['description'].sample().values[0]
prueba2 = test['description'].sample().values[0]

In [58]:
print(prueba1)
print(prueba2)

Se vende Apartamento en Excelente estado de conservación ubicado en Diamante II Bucaramanga.  Ubicado en el piso 3, torre de 5 pisos sin ascensor. Con un área privada de 65.30, y consta de 1 alcoba principal amplia con baño privado, una alcoba auxiliar, baño auxiliar, sala comedor, cocina integral y zona de
Codigo Inmueble 536 parqueadero cubierto, casa con excelentes acabados de 3 niveles, excelentes rutas de transporte.


In [67]:
import spacy
trained_nlp = spacy.load("../notebooks/assets/model/model-last")
# Perform the trained pipeline on this text
doc1 = trained_nlp(prueba1)
doc2 = trained_nlp(prueba2)
# We can display the predicted categories
print(doc1.cats)
print(doc2.cats)

{'1': 0.5, '0': 0.5}
{'1': 0.5, '0': 0.5}
